<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
به فیلم چند میدی؟
</font>
</h1>

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import keras
import math
import nltk

In [ ]:
from datasets import load_dataset, DatasetDict, load_from_disk
imdb_dataset = load_dataset("csv", data_files="train.csv", split='train').remove_columns('Movie_ID').train_test_split(0.20)
test_valid = imdb_dataset['test'].train_test_split(0.30)
imdb_dataset = DatasetDict({
    'train': imdb_dataset['train'],
    'test': test_valid['train'],
    'valid': test_valid['test']})
imdb_dataset

In [ ]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
from transformers import DistilBertTokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["Review"], padding=True, truncation=True)

In [ ]:
print(tokenize(imdb_dataset["train"][:2]))

In [ ]:
rating_encoded = imdb_dataset.map(tokenize, batched=True, batch_size=None)

In [ ]:
from transformers import AutoModel
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
def extract_hidden_states(batch):
# Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy().tolist()}

In [ ]:
rating_encoded.set_format("torch", columns=["input_ids", "attention_mask", "Rating"])
emotions_hidden = rating_encoded.map(extract_hidden_states, batched=True, batch_size=16)
emotions_hidden["train"].column_names
# emotion_hidden = load_from_disk("emotion-hidden")

In [ ]:
# def convert_to_numpy(length, train_test_valid, hidden_Rating):
#     X = []
#     for i in range(length):
#         X.append(np.array(emotions_hidden[train_test_valid][hidden_Rating][i].cpu().numpy()))
#     X= np.array(X)
#     return X

In [ ]:
# X_train = convert_to_numpy(len(emotions_hidden["train"]["hidden_state"]), "train", "hidden_state")
# y_train = convert_to_numpy(len(emotions_hidden["train"]["Rating"]), "train", "Rating")
# X_test = convert_to_numpy(len(emotions_hidden["test"]["hidden_state"]), "test", "hidden_state")
# y_test = convert_to_numpy(len(emotions_hidden["test"]["Rating"]), "test", "Rating")
# X_valid = convert_to_numpy(len(emotions_hidden["valid"]["hidden_state"]), "valid", "hidden_state")
# y_valid = convert_to_numpy(len(emotions_hidden["valid"]["Rating"]), "valid", "Rating")

In [ ]:
# df_train = pd.DataFrame(columns=['X_test', 'y_test'])
# df_train['X_train'] = X_train.tolist()
# df_train['y_train'] = y_train.tolist()
# df_train.to_csv('X,y_train.csv', index=False)

# df_test = pd.DataFrame(columns=['X_test', 'y_test'])
# df_test['X_test'] = X_test.tolist()
# df_test['y_test'] = y_test.tolist()
# df_test.to_csv('X,y_test.csv', index=False)


# df_valid = pd.DataFrame(columns=['X_valid', 'y_valid'])
# df_valid['X_valid'] = X_valid.tolist()
# df_valid['y_valid'] = y_valid.tolist()
# df_valid.to_csv('X,y_valid.csv', index=False)

In [ ]:
# #RELOADING
# df_train = pd.read_csv('X,y_train.csv')
# df_test = pd.read_csv('X,y_test.csv')
# df_valid = pd.read_csv('X,y_valid.csv')

In [ ]:
# X_train = df_train['X_train'].to_numpy()
# y_train = df_train['y_train'].to_numpy()
# X_test = df_test['X_test'].to_numpy()
# y_test = df_test['y_test'].to_numpy()
# X_valid = df_valid['X_valid'].to_numpy()
# y_valid = df_valid['y_valid'].to_numpy()

In [ ]:
# (X_train[2]).to_list()

In [ ]:
# fig, axes = plt.subplots(2, 5, figsize=(20,10))
# axes = axes.flatten()
# cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens", "pink", "YlOrBr", "YlGn", "YlGnBu"]
# labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# for i, (label, cmap) in enumerate(zip(labels, cmaps)):
#     df_emb_sub = df_emb.query(f"label == {i}")
#     axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap,
#     gridsize=20, linewidths=(0,))
#     axes[i].set_title(label)
#     axes[i].set_xticks([]), axes[i].set_yticks([])
# plt.tight_layout()
# plt.show()

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 10
model = (AutoModelForSequenceClassification
.from_pretrained(model_ckpt, num_labels=num_labels)
.to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import ClassLabel

# rating_encoded = rating_encoded.rename_column("Rating", "labels")
rating_encoded = rating_encoded.cast_column("labels", ClassLabel(num_classes=10))
rating_encoded['train']['labels']

In [ ]:
import torch

for split in rating_encoded.keys():
    rating_encoded[split] = rating_encoded[split].filter(
        lambda x: not torch.isnan(x["labels"]).any().item()  # keep only rows without NaN
    )

print(rating_encoded)


In [ ]:
rating_encoded = rating_encoded.map(lambda e: {"labels": int(e["labels"]) - 1})

In [ ]:
# Convert labels from tensors -> int
def to_int(example):
    return {"labels": int(example["labels"].item())}  # .item() gets scalar from tensor

rating_encoded = rating_encoded.map(to_int)
print(rating_encoded)


In [ ]:
rating_encoded['train']['labels'][0]

In [ ]:
rating_encoded["train"]['labels'][0] = rating_encoded["train"]['labels'][0].item()
rating_encoded["train"]['labels'][0]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 32
logging_steps = len(rating_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True,
    log_level="error")

from transformers import Trainer
trainer = Trainer(model=model, args=training_args,
compute_metrics=compute_metrics,
train_dataset=rating_encoded["train"],
eval_dataset=rating_encoded["valid"],
tokenizer=tokenizer)
trainer.train()

In [ ]:
# import matplotlib.pyplot as plt
# train_data["Rating"].value_counts(ascending=True).plot.barh()
# plt.title("Frequency of Classes")
# plt.show()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
max_len_sentence = max([len(s.split()) for s in train_dataset['Review']])   
max_len_word = 0
for review in (train_dataset['Review']):
    for word in review.split():
        if len(word) > max_len_word:
            max_len_word = len(word)
    
max_len_sentence, max_len_word

In [ ]:
# predict test samples
submission = pd.DataFrame()
submission
submission.to_csv('submission.csv', index=False)